In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import sys

sys.path.insert(0, '/home/storage2/hans/jax_reco/python')
from network import TriplePandleNet

In [3]:
net = TriplePandleNet('../../data/network/')

In [4]:
def transform_dimensions(dist, t_rho, t_z, km_scale = 1000):
    x0 = dist / km_scale
    x1 = np.cos(t_rho)
    x2 = np.sin(t_rho)
    x3 = t_z / km_scale
    return [x0, x1, x2, x3]

def dir_to_xyz(zenith, azimuth):
    z = np.cos(np.deg2rad(zenith))
    x = np.sin(np.deg2rad(zenith)) * np.cos(np.deg2rad(azimuth))
    y = np.sin(np.deg2rad(zenith)) * np.sin(np.deg2rad(azimuth))
    return [z, x, y]

In [5]:
dist = 25
z = -210
rho = 0.0
zenith = np.pi/2
azimuth = 0.0

x = np.concatenate([transform_dimensions(dist, rho, z), dir_to_xyz(zenith, azimuth)])

In [27]:
print(net.eval(x).shape)

(9,)


In [29]:
%timeit net.eval(x).block_until_ready()

74.6 µs ± 2.78 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [8]:
#from network import _eval_network
#params = net.get_network_params()
#print(params[0][0].shape, params[0][1].shape)
#%timeit _eval_network(x, params).block_until_ready()

In [30]:
import jax.numpy as jnp
batch_size = 500

xx = x[np.newaxis, :]
xx = np.repeat(xx, batch_size, axis=0)
y = np.random.normal(0.025, 0.001, 500)
xx[:, 0] = y
xx = jnp.array(xx)
print(xx.shape)

(500, 7)


In [31]:
%timeit net.eval_on_batch(xx).block_until_ready()

130 µs ± 5.6 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [32]:
z = net.eval_on_batch(xx)

In [33]:
print(z.shape)

(500, 9)
